 Agentic Sales Pipeline

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [5]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic Models for Structured Output

In [6]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Importing Tools

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [8]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [9]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

2025-02-09 05:45:05,279 - 140241096076160 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Creating Complete Sales Flow

In [10]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## Plotting the Flow

In [11]:
flow.plot()

Graph saved as crewai_flow_graph.html


In [12]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [13]:
emails = await flow.kickoff()

2025-02-09 05:46:27,122 - 140241096076160 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orchestration Platform
  - ICP: Enterprise companies looking int



# Agent: Lead Data Specialist
## Thought: Thought: I have gathered sufficient information about João Moura's personal details, as well as some information about Clearbit, including its industry, company size, and role relevance. However, I still need to determine the company's annual revenue and its market presence.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company profile including industry size revenue\"}"
## Tool Output: 

Search results: Title: Clearbit 2025 Company Profile: Valuation, Investors, Acquisition
Link: https://pitchbook.com/profiles/company/101066-86
Snippet: Information on acquisition, funding, cap tables, investors, and executives for Clearbit. Use the PitchBook Platform to explore the full profile.
---
Title: Leverage 100+ business data attributes - Clearbit
Link: https://clearbit.com/attributes
Snippet: Enrich your CRM and database, manage leads efficiently, and enable marketing personalization with over 100 business data po



# Agent: Lead Data Specialist
## Thought: Thought: Since I cannot access the previous link, I will check another source that might provide insights into Clearbit's market presence. I will choose the article that discusses the comparison between Clearbit and another market intelligence provider.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://happierleads.com/blog/clearbit-vs-zoominfo-market-intelligence-2023/\"}"
## Tool Output: 
Identify Anonymous Website Visitors - Personal Level IdentificationUPDATESCheckout our new updated PricingLearn morePricingAcademyHelp CenterBlogLog inSee your Web Visitors 100% FreeSee your Web Visitors 100% Free#1 AI Person-Level Website Visitor IdentityConvert Visitors to Qualified Leads.When the visitor is from 🇺🇸 get the exact visitor; when from 🇪🇺 access key decision-makers.See your Web Visitors 100% FreeNo credit card requiredRated 4.8/5 on Captera & G2ListenMuteTrusted by over 4,000 companies of all sizesFROM VISITOR 



# Agent: Lead Data Specialist
## Thought: Thought: I need to read the information available in the link that provides company size and some insights about Clearbit based on various company profiles.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.cbinsights.com/company/clearbit\"}"
## Tool Output: 
Clearbit - Products, Competitors, Financials, Employees, Headquarters LocationsTechnology VendorsRecent ResearchNewsletterMedia SupportLoginSearch company, investor...Who We ServeFunctionCorporate StrategyCorporate Venture CapitalVenture CapitalITCorporate InnovationHow VCs Use CB InsightsGo to the video gallery →IndustriesConsumer & RetailTechInsuranceHealthcareIndustrialsFinancial ServicesProfessional ServicesProfessional ServicesLearn More →Success StoriesHow We HelpUnderstand Tech MarketsAnticipate Competitor StrategyDetermine Tech StrategyConnect with Tech Vendors & PartnersTurbocharge Digital TransformationWhat We OfferPlatformOverviewTechnology S

...



# Agent: Cultural Fit Analyst
## Thought: Thought: I have found several relevant links that likely contain valuable information about Clearbit's company values, mission, and culture. I will read the most promising links to gather the necessary insights.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://clearbit.com/blog/company-values\"}"
## Tool Output: 
Defining Clearbit's company valuesClearbit customer loginBlog>Defining Clearbit's company valuesDefining Clearbit's company valuesMatt Sornson|August 12, 2019|4 minute readIn mid-2018, Clearbit was hitting an inflection point.
Since our founding in 2014, we'd been running as a small, hyper-efficient, and profitable business. Just about everyone in the company would weigh in on new hires, and every new employee spent significant time with the entire leadership team to understand what our values were. This organic model started to show some cracks as we started to scale.
At thirty-odd employees, Clearbit

 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfy the expect criteria, use the EXACT format below:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.




# Agent: Cultural Fit Analyst
## Final Answer: 
**Cultural Fit Assessment Report: Clearbit**

**Cultural Fit Score:** 9/10

**Cultural Values Overview:**
Clearbit has articulated its cultural values through a collaborative process that reflects the entire team rather than just leadership. The company's core values, which are integral to their culture, include:
- **Care (Give a shit)**: Empathy for customers and understanding their needs.
- **Craft (Master it)**: Commitment to continuous learning and improvement.
- **Team (Work together)**: Emphasis on 



# Agent: Lead Scorer and Validator
## Thought: Thought: Since I cannot access Crunchbase, I will try another relevant link, specifically the clearbit.com link that discusses their role in data enrichment services, which might provide essential information about the company's market presence and offerings.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://clearbit.com/\"}"
## Tool Output: 
Clearbit is Now Breeze Intelligence for HubSpotClearbit customer loginClearbit has joined HubSpot!The best B2B data provider is now part of the best B2B customer platform!Breeze Intelligence for HubSpotTo reach the right audiences and get the most out of AI, you need the right data. With Breeze Intelligence, you ‌get a full picture of your prospects and customers — making your marketing, sales, and service efforts even more effective.Check out Breeze!A data foundation you can build onWe gather public data from the web, our proprietary data, and the power of LLMs to con

2025-02-09 05:47:26,131 - 140241096076160 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Email Content Writer
## Task: Craft a highly personalized email using the lead's name, job title, company information, and any relevant personal or company achievements. The email should speak directly to the lead's interests and the needs of their company. This is not as cold outreach as it is a follow up to a lead form, so keep it short and to the point. Don't use any salutations or closing remarks, nor too complex sentences.
Our Company and Product: - Company Name: CrewAI - Product: Multi-Agent Orchestration Platform - ICP: Enterprise companies looking into Agentic automation. - Pitch: We are a platform that allows you to orchestrate AI Agents for automations to any vertical.
Use the following information: Personal Info: {'name': 'João Moura', 'job_title': 'Director of Engineering', 'role_relevance': 10, 'professional_background': None} Company Info: {'company_name': 'Clearbit', 'industry': 'B2B Data Enrichment', 'company_size': 1000, 'revenue': None, 'market_presence': 10}

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [14]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0181


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,120690,113783,6907,39


In [ ]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

## Inspecting Results

In [15]:
scores = flow.state["score_crews_results"]

In [16]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,João Moura
Job Title,Director of Engineering
Role Relevance,10
Professional Background,None
Company Name,Clearbit
Industry,B2B Data Enrichment
Company Size,1000
Revenue,None
Market Presence,10
Lead Score,90


## Results

In [17]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

Elevate your engineering processes with CrewAI's Multi-Agent Orchestration
Platform. Imagine streamlining your B2B data enrichment through seamless
automations tailored to your specific needs. Are you ready to maximize your AI-
driven initiatives? Schedule a quick 15-minute call to discuss how we can
implement effective AI agent orchestration for Clearbit.  Don't miss out on
optimizing your workflows. Click here to book your time now [Schedule
Meeting](#). Let’s unlock the potential of your data together!


## How Complex Can it Get?

In [18]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

## Plotting the Flow

In [19]:
flow = SalesPipeline()
flow.plot()

Graph saved as crewai_flow_graph.html


In [20]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)